In [1]:
import pandas as pd
import numpy as np 

import sys 
sys.path.append('../')

## Read datasets 

In [3]:
# mutual: indicated whether the link represents a mutual friend relation (1) or not (0)
# friend_date: Unix time stamp of when the friendship link was created
# user_id: anonymized unique id of a user
# friend_id: anonymized unique id of a user



# vote_date: Unix time stamp of the vote
# voter_id: anonymized unique id of the voter
# story_id: anonymized unique id of the story


friends = pd.read_csv('../../digg_friends.csv', names=['mutual', 'friend_date', 'user_id', 'friend_id' ])
votes = pd.read_csv('../../digg_votes1.csv', names=['vote_date', 'voter_id', 'story_id'])
# print(friends.duplicated().sum()) 

# #Friendship in both directions 

# mutual_0 = friends[friends['mutual'] == 0][['user_id', 'friend_id']]
# mutual_1 = friends[friends['mutual'] == 1][['user_id', 'friend_id']]

# mutual_2 = mutual_1.copy()
# mutual_2[['user_id', 'friend_id']] = mutual_2[['friend_id', 'user_id']]


# friends = pd.concat([mutual_0, mutual_1, mutual_2])[['user_id', 'friend_id']]
# friends = friends.drop_duplicates()

In [4]:
votes 

,vote_date,voter_id,story_id
0,1246573330,318,1
1,1246603391,399,1
2,1246573479,1749,1
3,1246599553,1964,1
4,1246568035,4362,1
...,...,...,...
3018192,1243937518,334596,3553
3018193,1243988005,334812,3553
3018194,1243909670,335028,3553
3018195,1243910982,335204,3553


## Removing 0 timestamps 

In [5]:
votes[votes['vote_date'] == '0'] 

,vote_date,voter_id,story_id


In [6]:
# user_ids = set(votes['voter_id'].to_list())

vote_freqs = dict(votes.groupby(['voter_id']).size())
filtered_users = list({k for k, v in vote_freqs.items() if v >= 20 } )



story_freqs = dict(votes.groupby(['story_id']).size())
filtered_stories = {k for k, v in story_freqs.items() if v >= 20 } 



filtered_votes = votes[votes['story_id'].isin(filtered_stories)]
filtered_votes = filtered_votes[filtered_votes['voter_id'].isin(filtered_users)]
print(len(filtered_users), ' users')

filtered_votes

27488  users


,vote_date,voter_id,story_id
0,1246573330,318,1
1,1246603391,399,1
2,1246573479,1749,1
3,1246599553,1964,1
5,1246570119,4610,1
...,...,...,...
3018189,1243955552,333991,3553
3018190,1243989870,334407,3553
3018192,1243937518,334596,3553
3018193,1243988005,334812,3553


In [237]:
# import networkx
# G = networkx.DiGraph()
# for u,f in zip(friends['user_id'].to_list(), friends['friend_id'].to_list()):
#     G.add_edge(u, f)
    
# G_filtered = G.subgraph(filtered_users)


# dummy = G_filtered.nodes
# users_ = filtered_votes['voter_id'].unique()

# all([d in users_ for  d in dummy])


# networkx.write_edgelist(G_filtered, 'filtered_friends.csv', data=False, delimiter='\t')
# filtered_votes [['voter_id', 'story_id', 'vote_date']].to_csv('./filtered_votes.csv', header=['voter_id', 'story_id', 'vote_date'], sep='\t', index=False)

# ## Users who voted > 100 
# voters = filtered_votes[['voter_id', 'vote_date']].groupby(['voter_id'])

# dict(voters.size())

In [4]:
filtered_friends = friends[friends['user_id'].isin(filtered_users) & friends['friend_id'].isin(filtered_users) ]
filtered_friends

,user_id,friend_id
937,336191,335404
941,336191,333584
948,336191,330208
952,336191,327421
953,336191,327306
...,...,...
1664605,173235,13360
1664665,78443,13360
1677306,302488,10740
1678324,161807,10678


In [5]:
# filtering friendships 
import random 
random.seed(1000)

all_users = set(filtered_friends['user_id'])
all_users_ = set(filtered_friends['friend_id'])


all_users = list(set(all_users).union(set(all_users_)))


# all_users = random.choices(list(all_users), k=1000)


print(len(all_users))


final_votes = filtered_votes[filtered_votes['voter_id'].isin(all_users)]
final_user_friends = filtered_friends[filtered_friends['user_id'].isin(all_users) & filtered_friends['friend_id'].isin(all_users) ]


7048


In [6]:
len(set(final_user_friends['user_id']).union(set(final_user_friends['friend_id'])))

7048

In [7]:
final_user_friends

,user_id,friend_id
937,336191,335404
941,336191,333584
948,336191,330208
952,336191,327421
953,336191,327306
...,...,...
1664605,173235,13360
1664665,78443,13360
1677306,302488,10740
1678324,161807,10678


In [8]:
final_user_friends [ [ 'user_id', 'friend_id']  ].to_csv('./output/digg_filtered_friends.txt', header=['user_id', 'friend_id'], sep='\t', index=False)

In [9]:
final_votes [['voter_id', 'story_id', 'vote_date']].to_csv('./output/digg_filtered_votes.txt', header=['voter_id', 'story_id', 'vote_date'], sep='\t', index=False)

In [ ]:
## Selecting a seed set 

In [33]:
# users with >100 votes 
active_voters = [k for k, v in dict(final_votes.groupby(['voter_id']).size()).items() if v >= 500]
len(active_voters)

571

In [36]:
active_users = [k for k, v in dict(final_user_friends.groupby(['user_id']).size()).items() if v >= 200]
len(active_users)

154

In [39]:
seed_set = set(active_users).intersection(set(active_voters))
seed_set

{13360,
 16255,
 24916,
 63501,
 68273,
 83820,
 87658,
 95610,
 101035,
 120100,
 124829,
 147259,
 147848,
 152331,
 152371,
 154329,
 154558,
 166025,
 193821,
 223113,
 223241,
 265784,
 268840,
 287433,
 291970,
 295716,
 314334,
 321735,
 322531,
 324532,
 332346}

## Old scripts 

In [ ]:
friends.shape, votes.shape

## Filter users who voted voted atleast on 20 stories

In [ ]:
users_friends = dict(friends.groupby(['user_id']).size())
friends_users = dict(friends.groupby(['friend_id']).size())

top_users = list({k:v for k,v in users_friends.items() if v < 400 and v > 20  })

len(top_users)
# top_users = set(top_users.keys())

# filtered_friends = friends[friends['user_id'].isin(top_users) & friends['friend_id'].isin(top_users)]
# print(len(top_users))

# filtered_friends[ [ 'user_id', 'friend_id']  ]

In [ ]:
# # total_users = top_followers
# # total_users = top_users
# total_users = list(set(list(top_users.keys()) + list(top_followers.keys())))

# filtered_friends = friends[(friends['user_id'].isin(total_users)) | (friends['friend_id'].isin(total_users))]

In [ ]:
friends[friends['user_id'].isin(top_users)]

In [ ]:
860 ** 2

In [ ]:
mutual_1

## Save files 

In [ ]:
# network_path: No header. SourceId	DestinationId
    
filtered_friends[ [ 'user_id', 'friend_id']  ].to_csv('./output/digg_filtered_friends.txt', header=['user_id', 'friend_id'], sep='\t', index=False)

In [ ]:
filtered_votes = votes[votes['voter_id'].isin(top_users)]

stories_count = filtered_votes.groupby(['story_id']).size()
top_stories = [k for k,v in stories_count.items() if v > 20]
filtered_votes = filtered_votes[votes['story_id'].isin(top_stories)]

users_counts = filtered_votes.groupby(['voter_id']).size()
top_users = [u for u, count in users_counts.items() if count > 20]
# print(len(top_users))
filtered_votes = filtered_votes[votes['voter_id'].isin(top_users)]
filtered_votes['voter_id'].unique().shape


filtered_votes.shape 

In [ ]:
filtered_votes['voter_id'].unique().shape 

## Filter by stories having >20 votes

In [ ]:
# stories = votes['story_id'].unique()
story_votes = dict(votes.groupby(['story_id']).size())

In [ ]:

story_votes = {k:v for k,v in story_votes.items() if v > 100 }

top_stories = list(story_votes.keys())

In [ ]:
filtered_votes = votes[votes['story_id'].isin(top_stories) & votes['voter_id'].isin(top_users)]

In [ ]:
len(top_users), len(top_stories)

In [ ]:
filtered_votes [['voter_id', 'story_id', 'vote_date']].to_csv('./output/digg_filtered_votes.txt', header=['voter_id', 'story_id', 'vote_date'], sep='\t', index=False)

In [ ]:
filtered_votes [['voter_id', 'story_id', 'vote_date']]


## Create network 



In [ ]:
import networkx

G = networkx.DiGraph()
for u in top_users: 
    G.add_node(u)
    friends = filtered_friends[filtered_friends['user_id'] == u]['friend_id']
    
    G.add_edges_from([(u, f) for f in friends])


# Load weights 

In [ ]:
# filtered_friends[['user_id', 'friend_id']]

filtered_friends[filtered_friends['user_id'] == 1230024]

In [ ]:
def prepare_weights(weight_file='./output/digg_with_weights.txt_InfProbs(10)', topic_col='Topic1'):

    weights = pd.read_csv(weight_file, sep='\t', 
                          names=['Influencer', 'Influenced'] + [f'Topic{i+1}' for i in range(10)], skiprows=1, index_col=False)

    data = weights[['Influencer', 'Influenced', topic_col]].to_dict('list')
    influencer, influenced, weights_ = data['Influencer'], data['Influenced'], data[topic_col]


    return {(influencer[i], influenced[i]): weights_[i] for i, _ in enumerate(weights_) }

# weight_dict = dict(zip(influenced, influenced), weights_)

In [ ]:
# dict([(i, j) for i, j in zip(influencers, influenced)], weights_
weights_2 = prepare_weights(topic_col='Topic2')

In [ ]:
from k_submodular.influence_maximization import weighted_network

weighted_G = weighted_network.weighted_network(G, 'rn')

In [ ]:
# weighted_G.edges()

In [ ]:
# pd.unique(filtered_friends[ [ 'user_id', 'friend_id']  ].values.ravel('K')).shape